## Лекция 2  BM5    

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re
import numpy as np
from math import log
import random

import nltk 
from nltk.text import Text 
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer() 

nltk.download("stopwords") 
#--------# 
from nltk.corpus import stopwords 
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import csv

with open('quora_question_pairs_rus.csv', 'r', encoding = 'utf-8') as f:  
    spamreader = csv.reader(f)
    data = [row[1:] for row in spamreader][1:]
    corpus = [row[1] for row in data]
    
corpus

['что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад',
 'как повысить скорость интернета путем взлома через dns',
 'найти остаток, когда математика 23 ^ 24 математика разделена на 24 23',
 'какая рыба выживет в соленой воде',
 'Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне',
 'что делает детей активными и далеки от телефонных и видеоигр',
 'что я должен делать, чтобы быть великим геологом?',
 'когда вы используете вместо',
 'как я могу взломать motorola dcx3400 для бесплатного интернета',
 'что некоторые технические специалисты могут рассказать о долговечности и надежности ноутбуков и их компонентов',
 'как я могу увидеть все мои комментарии к YouTube',
 'как вы можете легко научиться физике',
 'какой был ваш первый сексуальный опыт',
 'каковы законы об изменении вашего статуса от студенческой визы до зеленой карты в нас, как они сравниваются с иммиграционными законами в Японии',
 'как президентство козыря повлияет на учеников в н

In [23]:
### реализуйте эту функцию ранжирования 
from math import log

k = 2.0
b = 0.75
lengths = [len(doc) for doc in corpus]
avgdl = sum(lengths)/len(lengths)
N = len(corpus)

def q_score(q, N, doc, len_doc):        #подсчёт score отдельного слова
    n = np.count_nonzero(result1[q])
    tf = len(re.findall(' '+q+' ', doc))
    idf = IDF(N, n)
    score = idf*((tf*(k+1))/(tf+k*(1-b+b*len_doc/avgdl)))
    return score
    
def bm25(query, doc) -> float:   #query подать уже нормализованную))
    doc_contents = doc.split(' ')
    len_doc = len(doc_contents)
    scores = []
    for q in query:
        try:
            scores.append(q_score(q, N, doc, len_doc))
        except KeyError:
            scores.append(0)
    return sum(scores)

def IDF(N, n):   # qi - одно слово из запроса
    idf = log((N-n+0.5)/(n+0.5))
    return idf

In [4]:
def preprocess_query(raw_query):
    normalized = [pmm.normal_forms(x)[0] for x in raw_query.split() if x not in russian_stopwords]
    return normalized

for_queries = [row[0] for row in data[:1000]]
raw_query = random.choice(for_queries)
QUERY = preprocess_query(raw_query)
print(QUERY)

['волшебство', 'волшебник', 'научно', 'проверить']


In [5]:
def preprocess_query(raw_query):
    normalized = [pmm.normal_forms(x)[0] for x in raw_query.split() if x not in russian_stopwords]
    return normalized

In [7]:
with open('lemmatized.txt', 'r', encoding='utf-8') as f:
    content = f.read()
    lems = content.split('\t')

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lems[:10000])
result1 = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names())

In [17]:
%%time
bm25_scores = [bm25(QUERY, doc) for doc in corpus]

Wall time: 7min 43s


In [18]:
output = sorted([(e,i) for i,e in enumerate(bm25_scores)], reverse = True)

In [20]:
for i in output[:5]:
    print(data[i[1]])       

['Каковы наилучшие способы проверки идеи запуска?', 'как я могу проверить и проверить мою идею запуска', '1']
['как вы проверяете статус pf онлайн?', 'Я не могу проверить свой баланс онлайн-счета в сети, как я могу проверить свой баланс в автономном режиме', '1']
['какова процедура проверки баланса pf', 'Я не могу проверить свой баланс онлайн-счета в сети, как я могу проверить свой баланс в автономном режиме', '1']
['может карма как концепция быть научно доказанной', 'карма научно доказана', '1']
['если у меня будет стыковочный рейс, они переведут мой зарегистрированный багаж с одного самолета на другой или мне придется его повторно проверить', 'мой полет от бенгалуру до чандигарха через дели бит, я хочу проверить в Дели, можно проверить в Дели с моим багажом', '0']


### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [24]:
QUERY2 = preprocess_query('рождественские каникулы')
bm25_scores = [bm25(QUERY2, doc) for doc in corpus]
output = sorted([(e,i) for i,e in enumerate(bm25_scores)], reverse = True)

In [25]:
for i in output[:5]:
    print(data[i[1]], i[0]) 

['кто может рассказать некоторые обычаи о рождественском дне', 'какие рождественские традиции вы и ваша семья имеете на рождественский вечер и рождественский день', '1'] 25.32994807580993
['какие из ваших любимых рождественских традиций вы начали с вашей семьи', 'какие рождественские традиции вы и ваша семья имеете на рождественский вечер и рождественский день', '1'] 25.32994807580993
['что такое рождество, как на острове Рождества', 'что такое рождественский остров', '0'] 22.5243132808425
['почему есть рождественский остров', 'что такое рождественский остров', '0'] 22.5243132808425
['какой лучший рождественский подарок для вас', 'какой лучший рождественский подарок', '0'] 22.5243132808425


### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

In [28]:
b15 = 0
b11 = 1

In [27]:
def q_score_b(q, N, doc, len_doc, b):        #подсчёт score отдельного слова
    n = np.count_nonzero(result1[q])
    tf = len(re.findall(' '+q+' ', doc))
    idf = IDF(N, n)
    score = idf*((tf*(k+1))/(tf+k*(1-b+b*len_doc/avgdl)))
    return score

def bm25_b(query, doc, b) -> float:   #query подать уже нормализованную))
    doc_contents = doc.split(' ')
    len_doc = len(doc_contents)
    scores = []
    for q in query:
        try:
            scores.append(q_score_b(q, N, doc, len_doc, b))
        except KeyError:
            scores.append(0)
    return sum(scores)


In [29]:
bm25_scores_b15 = [bm25_b(QUERY2, doc, b15) for doc in corpus]


In [30]:
output_b15 = sorted([(e,i) for i,e in enumerate(bm25_scores_b15)], reverse = True)

In [31]:
for i in output_b15[:5]:
    print(data[i[1]], i[0]) 

['кто может рассказать некоторые обычаи о рождественском дне', 'какие рождественские традиции вы и ваша семья имеете на рождественский вечер и рождественский день', '1'] 17.99038249822644
['какие из ваших любимых рождественских традиций вы начали с вашей семьи', 'какие рождественские традиции вы и ваша семья имеете на рождественский вечер и рождественский день', '1'] 17.99038249822644
['какой лучший рождественский подарок вы когда-либо давали', 'какой лучший рождественский подарок вы когда-либо получали', '0'] 11.993588332150962
['почему вы открываете подарки в канун Рождества и каково происхождение этой традиции?', 'какой лучший рождественский подарок вы когда-либо получали', '0'] 11.993588332150962
['что такое рождество, как на острове Рождества', 'что такое рождественский остров', '0'] 11.993588332150962


In [32]:
bm25_scores_b11 = [bm25_b(QUERY2, doc, b11) for doc in corpus]

In [34]:
output_b11 = sorted([(e,i) for i,e in enumerate(bm25_scores_b11)], reverse = True)
for i in output_b15[:5]:
    print(data[i[1]], i[0]) 

['кто может рассказать некоторые обычаи о рождественском дне', 'какие рождественские традиции вы и ваша семья имеете на рождественский вечер и рождественский день', '1'] 17.99038249822644
['какие из ваших любимых рождественских традиций вы начали с вашей семьи', 'какие рождественские традиции вы и ваша семья имеете на рождественский вечер и рождественский день', '1'] 17.99038249822644
['какой лучший рождественский подарок вы когда-либо давали', 'какой лучший рождественский подарок вы когда-либо получали', '0'] 11.993588332150962
['почему вы открываете подарки в канун Рождества и каково происхождение этой традиции?', 'какой лучший рождественский подарок вы когда-либо получали', '0'] 11.993588332150962
['что такое рождество, как на острове Рождества', 'что такое рождественский остров', '0'] 11.993588332150962
